In [2]:
import pandas as pd
import numpy as np
# pd.set_option('display.max_rows', 1000)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_colwidth', 1000)


text_file = "/home/coochie/vscode/learning-log/data/text_data/yogabar/August 2021 All Order Report (1).txt"
excel_file = "/home/coochie/vscode/learning-log/data/text_data/yogabar/Mapper (1).xlsx"

In [3]:
orders = pd.read_csv(text_file, sep='\t')
mapper  = pd.read_excel(excel_file)
orders.columns
# mapper.columns

/tmp/ipykernel_109666/4156448323.py:1: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv(text_file, sep='\t')


Index(['amazon-order-id', 'merchant-order-id', 'purchase-date',
       'last-updated-date', 'order-status', 'fulfillment-channel',
       'sales-channel', 'order-channel', 'url', 'ship-service-level',
       'product-name', 'sku', 'asin', 'number-of-items', 'item-status',
       'quantity', 'currency', 'item-price', 'item-tax', 'shipping-price',
       'shipping-tax', 'gift-wrap-price', 'gift-wrap-tax',
       'item-promotion-discount', 'ship-promotion-discount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'item-extensions-data', 'is-business-order', 'purchase-order-number',
       'price-designation', 'fulfilled-by', 'buyer-company-name',
       'buyer-cst-number', 'buyer-vat-number', 'buyer-tax-registration-id',
       'buyer-tax-registration-country', 'buyer-tax-registration-type',
       'customized-url', 'customized-page', 'is-heavy-or-bulky',
       'is-replacement-order', 'is-exchange-order', 'original-order-id',
       'is-amazo

In [14]:
def clean_orders(data):
    data['purchase-date'] = pd.to_datetime(data['purchase-date'], format='%Y-%m-%dT%H:%M:%S%z')
    data['date'] = pd.to_datetime(data['purchase-date'], format='%Y-%m-%dT%H:%M:%S%z').dt.date
    data['week'] = 'week-' + data['purchase-date'].dt.isocalendar().week.astype(str)
    data['sales'] = data['item-price'] * data['quantity']
    return data

order_data = clean_orders(orders)

In [16]:
# 1 st question


df_product_daily_sales = order_data.groupby(['product-name', 'date'])['sales'].agg(['sum', 'count']).reset_index()
df_product_daily_sales = df_product_daily_sales.sort_values(by='date')
df_product_daily_sales.columns = ['ProductName', 'Date', 'TotalSales', 'NumOrders']
df_product_daily_sales

,ProductName,Date,TotalSales,NumOrders
1679,Yogabar Quick Cooking Oats 900g | Premium Oats...,2021-07-31,558.0,2
1385,Yogabar Multigrain Energy Bars 380Gm Pack (38G...,2021-07-31,1600.0,4
848,Yogabar Chocolate Chunk Multigrain-Energy Bars...,2021-07-31,0.0,0
629,Yogabar Breakfast Protein Variety (Almond Coco...,2021-07-31,1160.0,2
2087,Yogabar Wholegrain Breakfast Muesli - Dark Cho...,2021-07-31,0.0,0
...,...,...,...,...
284,"Yogabar 20G Protein Almond Bar - Pack of 12, K...",2021-08-31,2400.0,2
2214,Yogabar Wholegrain Breakfast Muesli Combo | Fr...,2021-08-31,33496.0,41
800,Yogabar Breakfast Protein-Bar Blueberry Pie - ...,2021-08-31,500.0,1
227,Yogabar 20 gram Protein Bar Chocolate Brownie ...,2021-08-31,8985.0,15


In [17]:
# 2nd question

# Top 10 products for the month with week on week sales trend views
df_top_10_products = order_data.groupby('product-name')['sales'].sum().reset_index()
df_top_10_products = df_top_10_products.nlargest(10, 'sales')
df_top_10_products.columns = ['ProductName', 'Sales']
df_top_10_products

,ProductName,Sales
88,Yogabar Multigrain Energy Bars 380Gm Pack (38G...,1438145.0
17,Yogabar 20 gram Protein Bar Variety Box - 6 x ...,1220319.0
127,"Yogabar Super Muesli, No Added or Hidden Sugar...",934162.0
14,Yogabar 20 Gram Protein Bar Variety Box - 6 x ...,922221.0
145,Yogabar Wholegrain Breakfast Muesli Combo | Fr...,916735.0
57,Yogabar Crunchy Peanut Butter 1kg | Dark Choco...,792967.0
77,Yogabar Dark Chocolate Peanut Butter| Creamy &...,548157.0
16,Yogabar 20 gram Protein Bar Chocolate Brownie ...,509183.0
7,Yogabar 100% Rolled Oats 1.2 kg | Premium Gold...,505200.0
38,Yogabar Breakfast Protein Variety (Almond Coco...,469940.0


In [18]:
# Create a week on week sales trend view
df_top_10_products_weekly_sales = order_data.groupby(['product-name','week' ])['sales'].sum().reset_index()
df_top_10_products_weekly_sales = df_top_10_products.merge(df_top_10_products_weekly_sales, left_on='ProductName' , right_on='product-name')
df_top_10_products_weekly_sales.drop(['product-name'],axis=1,inplace=True)
df_top_10_products_weekly_sales.columns = ['ProductName', 'TotalSalesOfMonth', 'WeekOfYear', 'WeeklySales']
df_top_10_products_weekly_sales

,ProductName,TotalSalesOfMonth,WeekOfYear,WeeklySales
0,Yogabar Multigrain Energy Bars 380Gm Pack (38G...,1438145.0,week-30,40760.0
1,Yogabar Multigrain Energy Bars 380Gm Pack (38G...,1438145.0,week-31,321680.0
2,Yogabar Multigrain Energy Bars 380Gm Pack (38G...,1438145.0,week-32,292800.0
3,Yogabar Multigrain Energy Bars 380Gm Pack (38G...,1438145.0,week-33,427740.0
4,Yogabar Multigrain Energy Bars 380Gm Pack (38G...,1438145.0,week-34,277365.0
5,Yogabar Multigrain Energy Bars 380Gm Pack (38G...,1438145.0,week-35,77800.0
6,Yogabar 20 gram Protein Bar Variety Box - 6 x ...,1220319.0,week-30,22762.0
7,Yogabar 20 gram Protein Bar Variety Box - 6 x ...,1220319.0,week-31,205883.0
8,Yogabar 20 gram Protein Bar Variety Box - 6 x ...,1220319.0,week-32,286650.0
9,Yogabar 20 gram Protein Bar Variety Box - 6 x ...,1220319.0,week-33,384883.0


In [ ]:
# 3rd question


# Category Wise day on day sales and week on week sales
df_category_daily_sales = order_data.merge(mapper, on='sku')[['Category', 'date', 'item-price']].groupby(['Category', 'date'])['item-price'].sum().reset_index()
df_category_daily_sales = df_category_daily_sales.sort_values(by='date')
df_category_daily_sales.columns = ['Category', 'Date','ItemPrice']
df_category_daily_sales

In [ ]:
df_category_weekly_sales = order_data.merge(mapper, on='sku')[['Category', 'purchase-date', 'item-price' ,'week']].groupby(['Category', 'week'])['item-price'].sum().reset_index()
df_category_weekly_sales.columns = ['Category', 'Week','ItemPrice']

df_category_weekly_sales

In [ ]:
# 4th question


# Product wise discount % on MRP Sales for the month
df_product_discount = order_data.merge(mapper, on='sku')[['product-name', 'item-price', 'MRP']].groupby('product-name').apply(lambda x: ((x['MRP'] - x['item-price']) / x['MRP']).mean() * 100).round(2).reset_index()
df_product_discount.columns = ['ProductName', 'Discount (%)']
df_product_discount